<a href="https://colab.research.google.com/github/pirjo2/programmeerimine-2025/blob/main/teemad/python/Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sissejuhatus Pandase ja Matplotlibi kasutamisse

## Mis on Pandas?
Pandas on populaarne Python'i teek, mis võimaldab mugavalt töödelda tabelandmeid. Seda kasutatakse andmeteaduses, analüüsis ja visualiseerimises. Pandas teeb andmete puhastamise, filtreerimise, sorteerimise ja analüüsi lihtsaks.

Pandas ja Matplotlib on olulised Python'i teegid andmete töötlemiseks ja visualiseerimiseks. Andmeteaduses on tähtis osata andmeid tõhusalt hallata ja tulemusi selgelt esitada.

## Andmeteaduse töövoog
Tüüpiline töövoog: andmete laadimine → puhastamine/ettevalmistus → analüüs → visualiseerimine → tulemuste esitamine. Pandast kasutatakse andmete haldamiseks ja analüüsiks, Matplotlibi visualiseerimiseks.

**Hea praktika:**
- Ära muuda originaalset DataFrame'i, vaid loo iga olulisema sammu jaoks uus muutuja (nt df_clean, df_sorted). Nii on töövoog jälgitav ja vigu lihtsam leida.
- Kui teed järjestikku mitu operatsiooni, kasuta Pandase .pipe() meetodit, mis võimaldab funktsioone järjestada ja koodi loetavust parandada.

Näide [pipe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html) kasutamisest:

df_clean = (
    df.dropna()
      .pipe(lambda d: d[d['age'] > 18])
      .sort_values('income', ascending=False)
)

## Pandase põhikontseptsioonid
Pandasel on kaks põhilist andmestruktuuri: **Series** (1-mõõtmeline) ja **DataFrame** (2-mõõtmeline).
- DataFrame on nagu arvutustabel: iga veerg on Series ja kõigil veergudel on sama indeks.

In [1]:
import pandas as pd

# Series loomine
s = pd.Series([1, 2, 3, 4])
print(s)

# Näidisandmestiku loomine DataFrame'ina
df = pd.DataFrame({
    'name': ['Anna', 'Jaan', 'Mari', 'Peeter', 'Liis'],
    'age': [22, 35, 19, 42, 28],
    'city': ['Tallinn', 'Tartu', 'Tallinn', 'Pärnu', 'Tartu'],
    'income': [1200, 2100, 950, 1800, 1600]
})
print(df)

0    1
1    2
2    3
3    4
dtype: int64
     name  age     city  income
0    Anna   22  Tallinn    1200
1    Jaan   35    Tartu    2100
2    Mari   19  Tallinn     950
3  Peeter   42    Pärnu    1800
4    Liis   28    Tartu    1600


## Andmetüübid Pandases
Pandas DataFrame'i iga veerg omab kindlat andmetüüpi, mis määrab, kuidas andmeid töödeldakse ja milliseid operatsioone saab teha. Olulised tüübid on:
- **object**: Tekst (stringid), segatüüpi väärtused. Näide: ['Tallinn', 'Tartu']
- **int**: Täisarvud. Näide: [1, 2, 3]
- **float**: Ujukomaarvud (komaga arvud). Näide: [3.14, 2.0]
- **bool**: Tõeväärtused (True/False). Näide: [True, False, True]
- **datetime64**: Kuupäevad ja kellaajad. Näide: ['2023-01-01', '2024-06-01']

Täisarvude puhul on Pandases mitmeid alamliike, mis määravad, kui palju mälu veerg kasutab:
- **Int16**: 16-bitine täisarv (-32768 kuni 32767). Sobib väikeste arvude jaoks, säästab mälu. Lubab puuduvaid väärtusi (pandas.NA).
- **Int32**: 32-bitine täisarv (-2 147 483 648 kuni 2 147 483 647). Kasutatakse suuremate arvude jaoks. Lubab puuduvaid väärtusi (pandas.NA).

Erinevalt NumPy int tüüpidest lubavad Pandase Int16 ja Int32 tüübid puuduvaid väärtusi, mis on kasulik näiteks ankeetandmete või mittetäielike andmestike puhul.
Kui tead, et veerus on ainult väikeseid arve, kasuta Int16, et vähendada mälukasutust. Suuremate arvude jaoks kasuta Int32 või Int64.

### Näited erinevatest andmetüüpidest Pandases:

In [2]:
import pandas as pd

df_types = pd.DataFrame({
    'nimi': ['Anna', 'Jaan'],  # object
    'vanus': pd.Series([22, pd.NA], dtype='Int16'),  # Int16 lubab puuduvaid väärtusi
    'palk': [1200.5, 2100.0],  # float
    'aktiivne': [True, False],  # bool
    'liitumise_kuup': pd.to_datetime(['2023-01-01', '2024-06-01'])  # datetime64
})
print(df_types.dtypes)
print(df_types)

# Tüübi muutmine Int32 peale (ka Int32 lubab puuduvaid väärtusi)
df_types['vanus'] = df_types['vanus'].astype('Int32')
print(df_types.dtypes)

nimi                      object
vanus                      Int16
palk                     float64
aktiivne                    bool
liitumise_kuup    datetime64[ns]
dtype: object
   nimi  vanus    palk  aktiivne liitumise_kuup
0  Anna     22  1200.5      True     2023-01-01
1  Jaan   <NA>  2100.0     False     2024-06-01
nimi                      object
vanus                      Int32
palk                     float64
aktiivne                    bool
liitumise_kuup    datetime64[ns]
dtype: object


Tüüpide kontroll ja muutmine on oluline, et vältida vigu ja optimeerida mälukasutust. Näiteks Int16 sobib väikeste arvude jaoks, Int32 suuremate jaoks. Kuupäevadega töötamiseks kasuta datetime64 tüüpi. Puuduvate väärtuste jaoks kasuta Pandase Int-tüüpe, mitte NumPy int-tüüpe.

## Indekseerimine ja indeksi muutmine
Igal DataFrame'il on indeks, mis määrab ridade järjestuse ja identiteedi. Vaikimisi on indeks numbriline (0, 1, 2, ...), kuid tihti on kasulik kasutada tähenduslikku indeksit, näiteks nime, kuupäeva või unikaalset koodi.

**Miks määrata eraldi indeks?**
- Kiirem ja mugavam andmete otsimine (nt df.loc['Anna'])
- Loogilisem andmete struktuur, eriti kui ridadel on unikaalne tunnus
- Grupitöötlus ja joinimine teiste andmestikega on lihtsam
- Ajalooliste andmete puhul saab kasutada kuupäeva indeksina

Kui ekspordid andmed Excelisse või Parquet-faili ja kasutad index=False, siis indeksit ei salvestata. Selle vältimiseks tee indeksist tulp tagasi.

In [3]:
# Indeksi määramine veeru põhjal
df_indexed = df.set_index('name')
print(df_indexed)

# Andmete otsimine indeksi abil
print(df_indexed.loc['Anna'])

# Indeksist tulba tegemine (enne eksporti)
df_reset = df_indexed.reset_index()
print(df_reset)

        age     city  income
name                        
Anna     22  Tallinn    1200
Jaan     35    Tartu    2100
Mari     19  Tallinn     950
Peeter   42    Pärnu    1800
Liis     28    Tartu    1600
age            22
city      Tallinn
income       1200
Name: Anna, dtype: object
     name  age     city  income
0    Anna   22  Tallinn    1200
1    Jaan   35    Tartu    2100
2    Mari   19  Tallinn     950
3  Peeter   42    Pärnu    1800
4    Liis   28    Tartu    1600


## Andmete sorteerimine
Sorteerimine aitab leida suurimaid/väiksemaid väärtusi või järjestada andmeid loogiliselt.

### Sorteerimine ühe või mitme tulba järgi
Saad sorteerida ühe või mitme tulba järgi, määrates tulbanimed listina. Näiteks vanuse ja sissetuleku järgi:

In [4]:
# Sorteeri vanuse järgi kahanevalt
df_sorted = df.sort_values('age', ascending=False)
print(df_sorted)

# Sorteeri vanuse ja sissetuleku järgi
df_multi_sorted = df.sort_values(['age', 'income'], ascending=[True, False])
print(df_multi_sorted)

     name  age     city  income
3  Peeter   42    Pärnu    1800
1    Jaan   35    Tartu    2100
4    Liis   28    Tartu    1600
0    Anna   22  Tallinn    1200
2    Mari   19  Tallinn     950
     name  age     city  income
2    Mari   19  Tallinn     950
0    Anna   22  Tallinn    1200
4    Liis   28    Tartu    1600
1    Jaan   35    Tartu    2100
3  Peeter   42    Pärnu    1800


### Sorteerimine custom funktsiooniga
Alates Pandas 1.1 saab kasutada sort_values(key=...), kus key on funktsioon (vaata ka [lambda funktsioone](https://docs.python.org/3/reference/expressions.html#lambda)), mis rakendatakse sorteeritavale veerule. Näiteks sorteerida nime pikkuse järgi.

In [5]:
# Sorteeri nime pikkuse järgi (custom funktsiooniga, Pandas >=1.1)
df_custom_sorted = df.sort_values('name', key=lambda x: x.str.len())
print(df_custom_sorted)

     name  age     city  income
0    Anna   22  Tallinn    1200
1    Jaan   35    Tartu    2100
2    Mari   19  Tallinn     950
4    Liis   28    Tartu    1600
3  Peeter   42    Pärnu    1800


## Uute veergude lisamine ja arvutamine
Sageli on vaja arvutada uusi veerge olemasolevate põhjal, näiteks arvutada netosissetulek, määrata vanusekategooria või luua bool-tunnus (True/False). Uued veerud aitavad andmeid paremini analüüsida ja visualiseerida.

Uue veeru saab lisada lihtsalt, omistades DataFrame'ile uue tulbanime:
- Arvutatud veerg: df['income_tax'] = df['income'] * 0.24
- Kategooria: df['age_group'] = pd.cut(df['age'], bins=[0,18,65,100], labels=['alaealine','täiskasvanu','eakas'])
- Bool-tunnus: df['is_adult'] = df['age'] >= 18

## Tabelite liitmine (joinimine)
Tabelite liitmine (merge/join) võimaldab ühendada andmeid erinevatest allikatest. Joinimiseks peab olema:
- Ühine veerg (või indeks), mille alusel liita
- Veerud peavad olema sama andmetüübiga
- Soovitavalt unikaalsed väärtused liitmisveerus vähemalt ühes tabelis

Kõige levinum meetod on pd.merge():

In [6]:
# Uue veeru loomine
df['income_tax'] = df['income'] * 0.24  # Näiteks tulumaksu arvutamine
df['is_adult'] = df['age'] >= 18 # tekitab boolean veeru: True/False
print(df)

# Tabelite liitmine (joinimine)
df_extra = pd.DataFrame({
    'name': ['Anna', 'Jaan', 'Mari'],
    'hobby': ['jooga', 'matkamine', 'lugemine']
})
df_joined = pd.merge(df, df_extra, on='name', how='left')
print(df_joined)

     name  age     city  income  income_tax  is_adult
0    Anna   22  Tallinn    1200       288.0      True
1    Jaan   35    Tartu    2100       504.0      True
2    Mari   19  Tallinn     950       228.0      True
3  Peeter   42    Pärnu    1800       432.0      True
4    Liis   28    Tartu    1600       384.0      True
     name  age     city  income  income_tax  is_adult      hobby
0    Anna   22  Tallinn    1200       288.0      True      jooga
1    Jaan   35    Tartu    2100       504.0      True  matkamine
2    Mari   19  Tallinn     950       228.0      True   lugemine
3  Peeter   42    Pärnu    1800       432.0      True        NaN
4    Liis   28    Tartu    1600       384.0      True        NaN


`pd.concat()` ühendab tabeleid kas ridade (`axis=0`) või veergude (`axis=1`) kaupa. Kasuta concat'i, kui soovid lihtsalt tabeleid järjestada või veerge kokku panna, mitte liita ühise veeru alusel nagu merge/join.

In [7]:
# Tabelite ühendamine (concat) ridade kaupa
df_part1 = df.iloc[:3]
df_part2 = df.iloc[3:]
df_concat = pd.concat([df_part1, df_part2], axis=0)
print(df_concat)

# Tabelite ühendamine (concat) veergude kaupa
df_cols1 = df[['name', 'age']]
df_cols2 = df[['city', 'income']]
df_concat_cols = pd.concat([df_cols1, df_cols2], axis=1)
print(df_concat_cols)

     name  age     city  income  income_tax  is_adult
0    Anna   22  Tallinn    1200       288.0      True
1    Jaan   35    Tartu    2100       504.0      True
2    Mari   19  Tallinn     950       228.0      True
3  Peeter   42    Pärnu    1800       432.0      True
4    Liis   28    Tartu    1600       384.0      True
     name  age     city  income
0    Anna   22  Tallinn    1200
1    Jaan   35    Tartu    2100
2    Mari   19  Tallinn     950
3  Peeter   42    Pärnu    1800
4    Liis   28    Tartu    1600


## Andmete inspekteerimine
Enne analüüsi on oluline oma andmeid tundma õppida.

In [8]:
# Esimeste ja viimaste ridade vaatamine
print(df.head())
print(df.tail())

     name  age     city  income  income_tax  is_adult
0    Anna   22  Tallinn    1200       288.0      True
1    Jaan   35    Tartu    2100       504.0      True
2    Mari   19  Tallinn     950       228.0      True
3  Peeter   42    Pärnu    1800       432.0      True
4    Liis   28    Tartu    1600       384.0      True
     name  age     city  income  income_tax  is_adult
0    Anna   22  Tallinn    1200       288.0      True
1    Jaan   35    Tartu    2100       504.0      True
2    Mari   19  Tallinn     950       228.0      True
3  Peeter   42    Pärnu    1800       432.0      True
4    Liis   28    Tartu    1600       384.0      True


`info` on üks kasulikumaid meetodeid, mis annab ülevaate veergude andmetüüpide ja puuduvatest väärtustest. See aitab mõista, milliseid andmeid on ja kas on vaja andmeid puhastada või teisendada.

In [9]:
# Veergude info ja puuduvate väärtuste kontroll
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        5 non-null      object 
 1   age         5 non-null      int64  
 2   city        5 non-null      object 
 3   income      5 non-null      int64  
 4   income_tax  5 non-null      float64
 5   is_adult    5 non-null      bool   
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 337.0+ bytes


`describe` annab statistilise kokkuvõtte arvuliste veergude kohta (nt keskmine, mediaan, min, max, kvartilid). See aitab mõista andmete jaotust ja leida võimalikke anomaaliaid. Kokkuvõte arvutatakse ainult arvuliste veergude kohta.

In [10]:
# Statistiline kokkuvõte
df.describe()

,age,income,income_tax
count,5.000000,5.000000,5.000000
mean,29.200000,1530.000000,367.200000
std,9.418068,460.434577,110.504299
min,19.000000,950.000000,228.000000
25%,22.000000,1200.000000,288.000000
50%,28.000000,1600.000000,384.000000
75%,35.000000,1800.000000,432.000000
max,42.000000,2100.000000,504.000000


## Veergude ja ridade valik ning filtreerimine
Veergude valimiseks kasuta `[[]]` notatsiooni: df[['name', 'age']] valib veerud 'name' ja 'age'. Ridade ja veergude täpsemaks valikuks kasuta `.loc[]` meetodit: df.loc[<indeks>, <veerg>].

- `[[]]` notatsioon valib veerge nime järgi, nt df[['name', 'age']].
- `.loc[]` võimaldab valida ridasid ja veerge indeksi ja veeru nime järgi, nt df.loc[0, 'name'] või df.loc[df['age'] > 18, ['name', 'age']].

Filtreerimiseks kasuta tingimusi, nt df[df['age'] > 18]. Tingimusi saab kombineerida (&, |).

### Iteratiivne filtreerimine pipe meetodiga
Kui soovid järjestikku mitut filtrit rakendada, kasuta pipe meetodit, mis muudab koodi loetavamaks.

In [11]:
# Veeru valimine [[]] notatsiooniga
alamvalik = df[['name', 'age']]
print(alamvalik)

# Ridade ja veergude valik .loc abil
valik = df.loc[df['age'] > 18, ['name', 'city']]
print(valik)

# Iteratiivne filtreerimine pipe meetodiga
df_filtered = (
    df.pipe(lambda d: d[d['age'] > 18])
      .pipe(lambda d: d[d['income'] > 1500])
      .pipe(lambda d: d[d['city'] == 'Tartu'])
)
print(df_filtered)

     name  age
0    Anna   22
1    Jaan   35
2    Mari   19
3  Peeter   42
4    Liis   28
     name     city
0    Anna  Tallinn
1    Jaan    Tartu
2    Mari  Tallinn
3  Peeter    Pärnu
4    Liis    Tartu
   name  age   city  income  income_tax  is_adult
1  Jaan   35  Tartu    2100       504.0      True
4  Liis   28  Tartu    1600       384.0      True


## Puuduvate andmete käsitlemine
Puuduvad andmed on tavalised ja nendega tuleb teadlikult ümber käia. Puuduvad väärtused võivad tekkida andmete kogumisel, vigade tõttu või seetõttu, et osa infot pole lihtsalt olemas.

**Levinumad praktikad puuduvate väärtuste käsitlemisel:**
- Puuduvate ridade eemaldamine (df.dropna()) – kasuta, kui puuduvus on juhuslik ja andmeid jääb piisavalt alles.
- Puuduvate väärtuste täitmine (df.fillna()) – kasuta, kui täitmine ei moonuta analüüsi (nt keskmise, mediaani, nulliga).
- Puuduvuse analüüs – vaata, kas puuduvus on juhuslik või süsteemne (nt ainult teatud grupis).
- Märgi puuduvad väärtused eraldi tunnusega, kui see on analüüsi jaoks oluline.

**Mida silmas pidada:**
- Kas puuduvus mõjutab analüüsi tulemusi?
- Kas täitmine või eemaldamine moonutab andmete jaotust?
- Kas eemaldamine vähendab oluliselt andmete hulka?
- Kas puuduvus on seotud mingi tunnusega (nt vanusegrupp, linn)?

Alati analüüsi puuduvate väärtuste mustrit enne otsuse tegemist!

In [12]:
# Puuduvate väärtuste arv veergude kaupa
print(df.isnull().sum())
# Ridade eemaldamine, kus on puuduvad andmed
df_puhas = df.dropna()
# Puuduvate väärtuste täitmine
df_taidetud = df.fillna(0)  # Või df.fillna(df.mean()) arvuliste veergude jaoks

name          0
age           0
city          0
income        0
income_tax    0
is_adult      0
dtype: int64


## Grupeerimine ja agregatsioon
Grupeerimine võimaldab andmeid jagada kategooriate kaupa ja rakendada igale grupile agregatsioonifunktsioone (nt keskmine, summa, min, max).

Agregatsioonifunktsioonid antakse Pandasele tavaliselt jutumärkides (nt 'mean'), sest need on Pandase sisseehitatud funktsioonide nimed. Võid kasutada ka Python funktsioone (nt np.mean, lambda vms).

**Levinumad agregatsioonifunktsioonid:**
- 'mean' – keskmine
- 'sum' – summa
- 'min' – miinimum
- 'max' – maksimum
- 'count' – ridade arv
- 'median' – mediaan
- 'std' – standardhälve

Agregatsioonifunktsioone saab kasutada nii stringina kui funktsioonina:
- df.groupby('city').agg({'income': 'mean'})
- df.groupby('city').agg({'income': np.mean})

### Custom funktsiooni kasutamine agregatsioonis
Võid defineerida oma funktsiooni ja kasutada seda groupby.agg sees. Näiteks arvutada veeru väärtuste vahe (max-min):

In [13]:
# Grupeerimine ja erinevad agregatsioonid

# Standardne agregatsioon
agg_df = df.groupby('city').agg({'age': 'mean', 'income': 'sum'})
print(agg_df)

# Mitme agregatsioonifunktsiooni kasutamine
agg_multi = df.groupby('city').agg({'income': ['mean', 'min', 'max', 'std']})
print(agg_multi)

# Custom funktsioon: vahe (max-min)
def vahe(x):
    return x.max() - x.min()
agg_custom = df.groupby('city').agg({'income': vahe})
print(agg_custom)

          age  income
city                 
Pärnu    42.0    1800
Tallinn  20.5    2150
Tartu    31.5    3700
         income                        
           mean   min   max         std
city                                   
Pärnu    1800.0  1800  1800         NaN
Tallinn  1075.0   950  1200  176.776695
Tartu    1850.0  1600  2100  353.553391
         income
city           
Pärnu         0
Tallinn     250
Tartu       500


`groupby` võimaldab igat gruppi eraldi töödelda. Saad iga grupi nime ja selle grupi alam-DataFrame'i, mida saab analüüsida, salvestada või visualiseerida.

In [14]:
# Prindi iga grupi alam-DataFrame
for city, group_df in df.groupby('city'):
    print(f'Grupi nimi: {city}')
    print(group_df)
    print('---')

Grupi nimi: Pärnu
     name  age   city  income  income_tax  is_adult
3  Peeter   42  Pärnu    1800       432.0      True
---
Grupi nimi: Tallinn
   name  age     city  income  income_tax  is_adult
0  Anna   22  Tallinn    1200       288.0      True
2  Mari   19  Tallinn     950       228.0      True
---
Grupi nimi: Tartu
   name  age   city  income  income_tax  is_adult
1  Jaan   35  Tartu    2100       504.0      True
4  Liis   28  Tartu    1600       384.0      True
---


## Pivot-tabelid Pandases

Pivot-tabelid võimaldavad andmeid ümber korraldada ja kokku võtta, et leida seoseid erinevate tunnuste vahel. Pivot on eriti kasulik, kui soovid näha, kuidas mingi tunnus (nt keskmine sissetulek) muutub erinevate kategooriate lõikes (nt linn ja vanusegrupp).

Pandases saab pivotit teha kahel viisil:

1. **`pivot_table`** – võimaldab arvutada agregaatväärtusi (nt keskmine, summa) ja sobib olukordadesse, kus ridade/veerude kombinatsioonid pole unikaalsed.
2. **`pivot`** – lihtne ümberkorraldus, kus igal ridade/veerude kombinatsioonil peab olema täpselt üks väärtus (st andmed peavad olema "täielikud").

### Näide 1: `pivot_table` (agregatsiooniga)

Oletame, et soovid näha iga linna ja vanusegrupi keskmist sissetulekut:

In [15]:
# Tekita vanusegrupid uue veeruna
df['age_group'] = pd.cut(df['age'], bins=[0,22,65,100], labels=['noor','keskealine','eakas'])
# pivot vanusegruppide ja linnade kaupa keskmine sissetulek
pivot_df = pd.pivot_table(df, values='income', index='city', columns='age_group', aggfunc='mean')

print(pivot_df)

age_group    noor  keskealine
city                         
Pärnu         NaN      1800.0
Tallinn    1075.0         NaN
Tartu         NaN      1850.0


/tmp/ipython-input-961171772.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = pd.pivot_table(df, values='income', index='city', columns='age_group', aggfunc='mean')


Siin arvutatakse iga linna ja vanusegrupi kohta keskmine sissetulek. Tulemus on "tabel-tabelis" kujul, kus read on linnad ja veerud vanusegrupid.

### Näide 2: `pivot` (ilma agregatsioonita)

Kui andmed on sellised, et igal linna ja nime kombinatsioonil on täpselt üks sissetulek, saab kasutada pivot meetodit:

In [16]:

pivot_simple = df.pivot(index='city', columns='name', values='income')
print(pivot_simple)

name       Anna    Jaan    Liis   Mari  Peeter
city                                          
Pärnu       NaN     NaN     NaN    NaN  1800.0
Tallinn  1200.0     NaN     NaN  950.0     NaN
Tartu       NaN  2100.0  1600.0    NaN     NaN


Siin saad iga linna kohta näha, mis on iga inimese sissetulek. Kui mõni kombinatsioon puudub, tekib vastavasse lahtrisse NaN.

**Kokkuvõte:**
- Kasuta `pivot_table`, kui soovid arvutada agregaatväärtusi või kui andmed pole "täielikud".
- Kasuta `pivot`, kui igal kombinatsioonil on täpselt üks väärtus ja soovid lihtsalt andmeid ümber korraldada.

### Pivot-tabeli "unpivot" ehk muutmine tagasi pikaks tabeliks

Kui soovid pivot_simple tabeli muuta tagasi pikaks ("long format") tabeliks, kus igal real on city, name ja income, kasuta Pandase `melt` meetodit:

In [17]:
# Muutmine tagasi pikaks tabeliks
df_long = pd.melt(pivot_simple.reset_index(), id_vars='city', var_name='name', value_name='income')
print(df_long)

# tekkinud NaN väärtustega ridadest saame lahti näiteks nii:
df_no_nan = df_long.dropna(subset=['income'])
print(df_no_nan)

       city    name  income
0     Pärnu    Anna     NaN
1   Tallinn    Anna  1200.0
2     Tartu    Anna     NaN
3     Pärnu    Jaan     NaN
4   Tallinn    Jaan     NaN
5     Tartu    Jaan  2100.0
6     Pärnu    Liis     NaN
7   Tallinn    Liis     NaN
8     Tartu    Liis  1600.0
9     Pärnu    Mari     NaN
10  Tallinn    Mari   950.0
11    Tartu    Mari     NaN
12    Pärnu  Peeter  1800.0
13  Tallinn  Peeter     NaN
14    Tartu  Peeter     NaN
       city    name  income
1   Tallinn    Anna  1200.0
5     Tartu    Jaan  2100.0
8     Tartu    Liis  1600.0
10  Tallinn    Mari   950.0
12    Pärnu  Peeter  1800.0


Siin saad iga (city, name) kombinatsiooni kohta sissetuleku eraldi real. See on kasulik, kui soovid andmeid uuesti töödelda või visualiseerida pikemal kujul.

## Andmete lugemine ja salvestamine
Andmeid saab Pandases lugeda ja salvestada erinevatesse failiformaatidesse. Mõned levinumad näited:
- **CSV**: Universaalne, lihtne tekstiformaat, sobib hästi andmete vahetamiseks erinevate programmide vahel. Ei säilita andmetüüpe ega vormingut.
- **Excel (XLSX)**: Mugav kontoritööks, toetab mitut lehte ja vormingut, kuid pole nii tõhus suurte andmete jaoks.
- **Parquet**: Tõhus, kiire ja kompaktne binaarformaat, sobib suurte andmete salvestamiseks ja töötlemiseks. Säilitab andmetüübid.

CSV on kõige universaalsem ja sobib andmete jagamiseks erinevate platvormide vahel. Parquet on eelistatud suurte andmete ja analüüsi jaoks, Excel sobib kontoritööks ja väiksemate andmete puhul.

In [18]:
# Andmete salvestamine erinevatesse formaatidesse
df.to_csv('output.csv', index=False)
# df.to_excel('output.xlsx', index=False) # vajab openpyxl paketti, mida ei pruugi vaikimisi olla installitud
df.to_parquet('output.parquet')

In [20]:
# Andmete lugemine erinevatest failidest
df_csv = pd.read_csv('Islander_data.csv')
df_excel = pd.read_excel('Islander_data.xlsx')
#df_parquet = pd.read_parquet('Islander_data.parquet')

## Andmestiku kohta

Näidetes kasutame
[Kaggle](https://www.kaggle.com/datasets/steveahn/memory-test-on-drugged-islanders-data?resource=download)
"Memory Test on Drugged Islanders Data" andmestikku.

Andmestiku kirjeldus

Eksperiment, kus uuritakse ärevusvastaste ravimite mõju mälu taastamisele, kui osalejaid on eelnevalt suunatud meenutama rõõmsaid või kurbi mälestusi. Osalejateks on virtuaalsed 'Islanders', kes matkivad pärisinimeste käitumist välistele teguritele reageerides.

Uuritavad ravimid (tuntud kui) [Annus 1, 2, 3]:

A - Alprazolam (Xanax, pikaajaline) [1mg/3mg/5mg]

T - Triazolam (Halcion, lühiajaline) [0.25mg/0.5mg/0.75mg]

S - Suhkrutablett (platseebo) [1 tbl/2 tbl/3 tbl]

Andmete lugemiseks on kõige lihtsam kasutada `read_csv()` funktsiooni:

## Praktiline ülesanne: `Islander_data` andmestik

**Andmestiku tutvustus (eesti keeles):**
Islander_data andmestik sisaldab infot Islandi elanike kohta: vanus, sugu, elukoht, haridus, sissetulek, hobid ja muud tunnused. Andmestik sobib demograafiliseks analüüsiks, sissetulekute ja hariduse seoste uurimiseks ning andmetöötluse harjutamiseks Pandasega.

### Ülesande etapid
1. Laadi andmestik Pandase DataFrame'iks (CSV või Excel).
2. Inspekteeri andmestikku: vaata esimesi ridu, veergude tüüpe, puuduvate väärtuste arvu.
3. Filtreeri andmestik: leia kõik isikud, kelle vanus on üle 30 ja perekonnanimi on 'Durand'.
4. Millised on kõige levinumad perekonnanimed, kui palju neid on?
5. Kasutades kogu andmestikku, lisa uus veerg, mis näitab, kas isik on noor, keskealine või vanur (noor: 0-18, keskealine: 19-65, vanur: 66+).
6. Grupeeri andmed uue vanuse tulba järgi ja arvuta iga grupi keskmine vanus ja `diff` skoor.
7. Lisa gupeerimisse `Duug` veerg, milline ravim toimis paremini mälu parandamisel.


In [22]:
# lisa oma lahendus siia, võid lisada 'cell'-e nii palju kui vaja
df = pd.read_csv('Islander_data.csv')
df.head()

,first_name,last_name,age,Happy_Sad_group,Dosage,Drug,Mem_Score_Before,Mem_Score_After,Diff
0,Bastian,Carrasco,25,H,1,A,63.5,61.2,-2.3
1,Evan,Carrasco,52,S,1,A,41.6,40.7,-0.9
2,Florencia,Carrasco,29,H,1,A,59.7,55.1,-4.6
3,Holly,Carrasco,50,S,1,A,51.7,51.2,-0.5
4,Justin,Carrasco,52,H,1,A,47.0,47.1,0.1


In [23]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   first_name        198 non-null    object 
 1   last_name         198 non-null    object 
 2   age               198 non-null    int64  
 3   Happy_Sad_group   198 non-null    object 
 4   Dosage            198 non-null    int64  
 5   Drug              198 non-null    object 
 6   Mem_Score_Before  198 non-null    float64
 7   Mem_Score_After   198 non-null    float64
 8   Diff              198 non-null    float64
dtypes: float64(3), int64(2), object(4)
memory usage: 14.1+ KB


,age,Dosage,Mem_Score_Before,Mem_Score_After,Diff
count,198.000000,198.000000,198.000000,198.000000,198.000000
mean,39.530303,1.989899,57.967677,60.922222,2.954545
std,12.023099,0.818504,15.766007,18.133851,10.754603
min,24.000000,1.000000,27.200000,27.100000,-40.400000
25%,30.000000,1.000000,46.525000,47.175000,-3.175000
50%,37.000000,2.000000,54.800000,56.750000,1.700000
75%,48.000000,3.000000,68.400000,73.250000,5.925000
max,83.000000,3.000000,110.000000,120.000000,49.000000


In [25]:
df_filter = df[(df['age'] > 30) & (df['last_name'] == 'Durand')]
df_filter.head()

,first_name,last_name,age,Happy_Sad_group,Dosage,Drug,Mem_Score_Before,Mem_Score_After,Diff
6,Ava,Durand,35,S,1,A,44.1,56.0,11.9
7,Jamie,Durand,38,H,1,A,76.3,74.8,-1.5
9,Mark,Durand,36,S,1,A,54.8,75.9,21.1
10,Maximiliano,Durand,63,S,1,A,90.0,102.0,12.0
29,Darren,Durand,36,H,2,A,74.9,70.8,-4.1


In [28]:
pop_perekonnainimi = df['last_name'].value_counts()
print("Populaarsemad perekonnanimed:")
print(pop_perekonnainimi)

Populaarsemad perekonnanimed:
last_name
Durand       44
Carrasco     43
Lopez        23
Takahashi    20
Summers      12
Steiner      12
Kennedy      10
McCarthy      9
Gonzalez      6
Bernard       3
Novak         3
Connolly      3
Rodriguez     2
Price         2
Castro        2
Hajek         2
Morin         1
Fiala         1
Name: count, dtype: int64


In [31]:
df['age_group'] = pd.cut(df['age'], bins=[0, 18, 65, 100], labels=['noor', 'keskealine', 'vanur'], right=True)
print(df[['age', 'age_group']].head())

   age   age_group
0   25  keskealine
1   52  keskealine
2   29  keskealine
3   50  keskealine
4   52  keskealine


In [33]:
vanuste_grupid = df.groupby('age_group').agg({'age': 'mean', 'Diff': 'mean'})
print(vanuste_grupid)

                  age      Diff
age_group                      
noor              NaN       NaN
keskealine  38.147368  2.996842
vanur       72.375000  1.950000


/tmp/ipython-input-77301540.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  vanuste_grupid = df.groupby('age_group').agg({'age': 'mean', 'Diff': 'mean'})


In [38]:
age_group_Drug = df.groupby(['age_group', 'Drug'], observed=True).agg({'Diff': 'mean'}).reset_index()
print("Keskmine Diff skoor vanusegrupi ja ravimi järgi:")
print(age_group_Drug)

Keskmine Diff skoor vanusegrupi ja ravimi järgi:
    age_group Drug       Diff
0  keskealine    A   9.281538
1  keskealine    S  -0.107812
2  keskealine    T  -0.442623
3       vanur    A  15.600000
4       vanur    S  -2.200000
5       vanur    T  -2.800000


Parima tulemuse saavutas vanurite grupp võttes ravimit A sooritades suurima erinuvuse skooriga 15,6.

## Mis edasi?
Kui oled Pandase põhitõed selgeks saanud, soovitan edasi uurida ametlikku dokumentatsiooni ja tutvuda Pandase õppetutorialidega:

- [Pandas dokumentatsioon](https://pandas.pydata.org/docs/)
- [Pandas intro-tutorialid](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html)

Sealt leiad põhjalikud näited, juhendid ja ülesanded, mis aitavad Pandase oskusi edasi arendada.